In [186]:
# semantics

import pandas as pd
import numpy as np
from sklearn import preprocessing, metrics

from sklearn.decomposition import PCA
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegressionCV

In [187]:
# initialization

train = pd.read_csv('./data_semantics_training.csv', na_values=['NULL', '', ' ', 0])
test = pd.read_csv('./data_sematics_test.csv', na_values=['NULL', '', ' ', 0])
train.head(1)

,serial_no,C1,C2,C3,C4,C5,C6,C7,C8
0,1,1,1,15200.0,80.0,6.5,24.0,Healthy,Live


In [188]:
xtrain = train.copy()
xtrain.drop(['serial_no'], inplace=True, axis=1)
xtrain.fillna(xtrain.mean(), inplace=True)
xtrain['C8'] = xtrain['C8'].fillna(value='missing')
one_hot = pd.get_dummies(xtrain['C8'])
xtrain.drop(['C8'], inplace=True, axis=1)
xtrain = xtrain.join(one_hot)
xtrain.head(2)

,C1,C2,C3,C4,C5,C6,C7,Closed,Live,missing
0,1,1,15200.0,80.0,6.50,24.0,Healthy,0,1,0
1,2,2,26100.0,90.0,6.75,30.0,Healthy,1,0,0


In [189]:
xtest = test.copy()
xtest.drop(['serial_no'], inplace=True, axis=1)
xtest.fillna(xtrain.mean(), inplace=True)
xtest['C8'] = xtest['C8'].fillna(value='missing')
one_hot = pd.get_dummies(xtest['C8'])
xtest.drop(['C8'], inplace=True, axis=1)
xtest = xtest.join(one_hot)
xtest.head(2)

,C1,C2,C3,C4,C5,C6,Closed,Live,missing
0,700,700,16800.000000,80.000000,6.500000,24.0,1,0,0
1,701,701,21584.496124,69.147287,5.675194,21.2,0,0,1


In [190]:
ytrain = xtrain[['C7']]
xtrain.drop(['C7'], inplace=True, axis=1)

enc = preprocessing.LabelEncoder()
enc.fit(ytrain.values)
ytrain = enc.transform(ytrain.values)

C:\Users\Lenovo\AppData\Roaming\Python\Python36\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Lenovo\AppData\Roaming\Python\Python36\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [191]:
pca = PCA()
pca.fit(xtrain)
pca_train = pca.transform(xtrain)
pca_test = pca.transform(xtest)

fulltrain = pd.concat([xtrain.reset_index(drop=True), pd.DataFrame(pca_train)], axis=1)
fulltest = pd.concat([xtest.reset_index(drop=True), pd.DataFrame(pca_test)], axis=1)

In [137]:
#LR = useless

mod = LogisticRegressionCV(cv=5, random_state=1, multi_class='ovr', max_iter=10000)
mod.fit(xtrain, ytrain)
pred = mod.predict(xtest)
pred = enc.inverse_transform(pred)

In [195]:
#GBC = rank 1

mod = OneVsRestClassifier(GradientBoostingClassifier())
mod.fit(xtrain, ytrain)
pred = mod.predict(xtest)
pred = enc.inverse_transform(pred)

In [ ]:
#GBC = rank 7

mod = OneVsOneClassifier(GradientBoostingClassifier())
mod.fit(xtrain, ytrain)
pred = mod.predict(xtest)
pred = enc.inverse_transform(pred)

In [205]:
#GBC = rank 1

mod = OneVsRestClassifier(xgb.XGBClassifier(learning_rate=0.01, n_estimators=100, 
                                            objective='multi:softmax', max_depth=6, num_class=4))
mod.fit(xtrain, ytrain)
pred = mod.predict(xtest)
pred = enc.inverse_transform(pred)

In [201]:
import xgboost as xgb

xg_train = xgb.DMatrix(xtrain, label=ytrain)
xg_test = xgb.DMatrix(xtest)
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.01
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 4
num_round=100

model = xgb.train(param, xg_train, num_round)
pred = model.predict(xg_test)
pred = enc.inverse_transform(pred.astype(int))

In [206]:
submission = pd.read_csv('./data_semantics_sample_random_submission.csv')
submission['C7'] = pred
submission.head(2)
submission.to_csv('submission.csv', index=False)